In [2]:
cd /Users/HaoShaochun/Documents/Study/DL-Models/transformers_/examples/distillation

/Users/HaoShaochun/Documents/Study/DL-Models/transformers_/examples/distillation


In [18]:
!head -2 data/dump.txt

It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .
Federal Master Trainer and Senior Instructor of the Italian Federation of Aerobic Fitness , Group Fitness , Postural Gym , Stretching and Pilates; from 2004 , he has been collaborating with Antiche Terme as personal Trainer and Instructor of Stretching , Pilates and Postural Gym .


## 数据准备

### 第一步

In [13]:
import pickle

In [14]:
with open("./data/binarized_text.bert-base-uncased.pickle", "rb") as f:
    data = pickle.load(f)

In [23]:
data[0]

array([  101,  2007,  1996,  3522,  2933,  1997, 14925,  2226,  2184,
        2213,  1010,  2057,  2031,  3621, 11792,  1996,  2862,  1997,
        1996, 22165,  1012,   102], dtype=uint16)

In [19]:
from transformers import BertTokenizer

I0427 11:46:11.011744 4378392000 file_utils.py:41] PyTorch version 1.3.1 available.
I0427 11:46:16.600965 4378392000 file_utils.py:57] TensorFlow version 2.1.0 available.


In [20]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

I0427 11:46:27.990487 4378392000 tokenization_utils.py:504] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/HaoShaochun/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [27]:
tokenizer.encode("leaving Master Trainer.")

[101, 2975, 3040, 10365, 1012, 102]

In [28]:
data[0]

array([  101,  2007,  1996,  3522,  2933,  1997, 14925,  2226,  2184,
        2213,  1010,  2057,  2031,  3621, 11792,  1996,  2862,  1997,
        1996, 22165,  1012,   102], dtype=uint16)

In [29]:
tokenizer.decode(_)

'[CLS] with the recent plan of ecu 10m, we have slightly enlarged the list of the ngos. [SEP]'

### 第二步

In [30]:
with open("./data/token_counts.bert-base-uncased.pickle", "rb") as f:
    count = pickle.load(f)

In [32]:
len(count)

30522

## 训练

In [38]:
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizer,
    DistilBertConfig,
    DistilBertForMaskedLM,
    DistilBertTokenizer,
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizer,
)

In [47]:
import numpy as np

In [39]:
student_config_class, student_model_class = DistilBertConfig, DistilBertForMaskedLM
teacher_config_class, teacher_model_class, teacher_tokenizer_class = \
BertConfig, BertForMaskedLM, BertTokenizer

In [40]:
teacher_name = "bert-base-uncased"
tokenizer = teacher_tokenizer_class.from_pretrained(teacher_name)

I0427 14:54:27.481127 4378392000 tokenization_utils.py:504] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/HaoShaochun/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [42]:
special_tok_ids = {}
for tok_name, tok_symbol in tokenizer.special_tokens_map.items():
    idx = tokenizer.all_special_tokens.index(tok_symbol)
    special_tok_ids[tok_name] = tokenizer.all_special_ids[idx]

In [41]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [43]:
special_tok_ids

{'unk_token': 100,
 'sep_token': 102,
 'pad_token': 0,
 'cls_token': 101,
 'mask_token': 103}

In [78]:
np.maximum(count, 1) ** -0.7

array([1., 1., 1., ..., 1., 1., 1.])

In [89]:
5 ** -0.7, 10 ** -0.7

(0.3241313193385525, 0.19952623149688797)

In [84]:
import torch

In [85]:
token_probs = np.maximum(count, 1) ** -0.7
for idx in special_tok_ids.values():
    token_probs[idx] = 0.0  # do not predict special tokens
token_probs = torch.from_numpy(token_probs)

In [86]:
token_probs

tensor([0., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64)

In [63]:
stu_architecture_config = DistilBertConfig.from_pretrained(
    "training_configs/distilbert-base-uncased.json")
stu_architecture_config.output_hidden_states = True
student = DistilBertForMaskedLM(stu_architecture_config)

I0427 15:06:02.958365 4378392000 configuration_utils.py:280] loading configuration file training_configs/distilbert-base-uncased.json
I0427 15:06:02.962847 4378392000 configuration_utils.py:318] Model config DistilBertConfig {
  "_num_labels": 2,
  "activation": "gelu",
  "architectures": null,
  "attention_dropout": 0.1,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "no_repeat_ngram_size": 0,
  "num_beams": 1,
  "num_return_sequences": 1,
  "output_attentions": fals

In [ ]:
teacher = BertForMaskedLM.from_pretrained("bert-base-uncased", output_hidden_states=True)

## Distiller

In [91]:
from lm_seqs_dataset import LmSeqsDataset

In [90]:
from torch.utils.data import BatchSampler, DataLoader, RandomSampler

In [239]:
class Args:
    def __init__(self):
        self.max_model_input_size = 512
        self.mlm = True
        self.special_tok_ids = special_tok_ids
        self.is_master = False
        self.fp16 = True
        self.vocab_size = 30522

In [240]:
args = Args()

### prepare batch mlm

In [121]:
dataset = LmSeqsDataset(params=args, data=data)

I0427 17:20:58.519951 4378392000 lm_seqs_dataset.py:68] Splitting 0 too long sequences.
I0427 17:20:58.524666 4378392000 lm_seqs_dataset.py:111] Remove 2 too short (<=11 tokens) sequences.
I0427 17:20:58.527251 4378392000 lm_seqs_dataset.py:127] Remove 0 sequences with a high level of unknown tokens (50%).


In [148]:
sampler = RandomSampler(dataset)
sampler = BatchSampler(sampler=sampler, batch_size=3, drop_last=False)

In [149]:
dataloader = DataLoader(dataset=dataset, batch_sampler=sampler, collate_fn=dataset.batch_sequences)

In [150]:
len(dataloader)

33

In [151]:
for batch in dataloader:
    break

In [152]:
token_ids, lengths = batch

In [153]:
attn_mask = torch.arange(token_ids.size(1), dtype=torch.long, device=lengths.device) < lengths[:, None]

In [167]:
token_ids.shape

torch.Size([3, 39])

In [400]:
attn_mask.shape

torch.Size([3, 39])

In [168]:
token_ids

tensor([[  101,  1999,  2755,  1010,  2009,  2003,  2035,  1037,  3160,  1997,
          2576,  2097,  1010,  2500,  2031,  2056,  2023,  1998,  1045,  2052,
          2066,  2000,  6911,  1996,  2391,  1010,  2498,  2590,  2097,  4148,
          2065,  2057,  2079,  2025,  2215,  2009,  2000,  1012,   102],
        [  101,  2101,  2006,  1010,  2009,  2003,  2825,  2000,  8752, 14472,
          2030,  4961,  1011,  2039,  2336,  1999,  2048,  3584,  9620,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2023,  2003,  2055,  2004,  2172,  2004,  2057,  2064,  3246,
          2005,  2306,  1996, 25722,  1997,  1996,  2691, 13424,  3343,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]])

In [166]:
attn_mask

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [358]:
attn_mask.shape

torch.Size([3, 39])

In [169]:
bs, max_seq_len = token_ids.size()
mlm_labels = token_ids.new(token_ids.size()).copy_(token_ids)

In [174]:
mlm_labels == token_ids

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True]])

In [180]:
token_probs

tensor([0., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64)

In [184]:
x_prob = token_probs[token_ids.flatten()]

In [185]:
x_prob

tensor([0.0000, 0.0613, 1.0000, 0.0337, 0.1149, 0.0686, 0.2853, 0.0814, 1.0000,
        0.0496, 0.4635, 0.1661, 0.0337, 1.0000, 0.2333, 1.0000, 0.1051, 0.0527,
        0.1273, 0.2561, 0.2561, 0.0550, 1.0000, 0.0303, 1.0000, 0.0337, 1.0000,
        0.4635, 0.1661, 1.0000, 0.1995, 0.1502, 0.6156, 0.1273, 0.6156, 0.1149,
        0.0550, 0.0382, 0.0000, 0.0000, 0.4635, 0.1322, 0.0337, 0.1149, 0.0686,
        0.6156, 0.0550, 1.0000, 1.0000, 0.2148, 1.0000, 0.0784, 0.3241, 1.0000,
        0.0613, 0.3241, 1.0000, 1.0000, 0.0382, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1051, 0.0686,
        0.4635, 0.1436, 1.0000, 0.1436, 0.1502, 0.2333, 1.0000, 0.1081, 1.0000,
        0.0303, 1.0000, 0.0496, 0.0303, 0.6156, 0.4635, 0.6156, 0.0382, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [187]:
# 15% 的 mask
import math
n_tgt = math.ceil(0.15 * lengths.sum().item())

In [188]:
n_tgt

13

In [191]:
tgt_ids = torch.multinomial(x_prob / x_prob.sum(), n_tgt, replacement=False)

In [192]:
tgt_ids

tensor([40, 48, 13, 81, 34, 82, 47, 10, 29, 53, 96, 83, 86])

In [220]:
pred_mask = torch.zeros(bs * max_seq_len, dtype=torch.bool, device=token_ids.device)

In [225]:
pred_mask[tgt_ids] = 1

In [227]:
pred_mask = pred_mask.view(bs, max_seq_len)

In [228]:
pred_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
          True, False, False,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,  True,
         False, False, False, False,  True, False, False, False, False],
        [False,  True, False, False, False, False, False, False,  True,  True,
         False, False, False, False,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [False, False, False,  True,  True,  True, False, False,  True, False,
         False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [231]:
pred_mask[token_ids == args.special_tok_ids["pad_token"]] = 0

In [232]:
pred_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
          True, False, False,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,  True,
         False, False, False, False,  True, False, False, False, False],
        [False,  True, False, False, False, False, False, False,  True,  True,
         False, False, False, False,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [False, False, False,  True,  True,  True, False, False,  True, False,
         False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [234]:
_token_ids_real = token_ids[pred_mask]

In [255]:
_token_ids_real

tensor([ 2576,  2500,  4148,  2215,  2101,  8752, 14472,  2336,  2055,  2004,
         2172,  2064,  3343])

In [265]:
_token_ids_rand = _token_ids_real.clone().random_(args.vocab_size)
_token_ids_rand

tensor([ 2306,  1111, 29427, 27823, 16740, 26188, 16092, 18823, 25777, 22342,
        17539, 20330, 11371])

In [242]:
_token_ids_mask = _token_ids_real.clone().fill_(args.special_tok_ids["mask_token"])
_token_ids_mask

tensor([103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103, 103])

In [245]:
# parser.add_argument("--word_mask", default=0.8, type=float, help="Proportion of tokens to mask out.")
# parser.add_argument("--word_keep", default=0.1, type=float, help="Proportion of tokens to keep.")
# parser.add_argument("--word_rand", default=0.1, type=float, help="Proportion of tokens to randomly replace.")
pred_probs = torch.FloatTensor([0.8, 0.1, 0.1])
pred_probs

tensor([0.8000, 0.1000, 0.1000])

In [246]:
probs = torch.multinomial(pred_probs, len(_token_ids_real), replacement=True)
probs

tensor([0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0])

In [275]:
_token_ids = (
    _token_ids_mask * (probs == 0).long()
    + _token_ids_real * (probs == 1).long()
    + _token_ids_rand * (probs == 2).long()
)
_token_ids

tensor([  103,   103, 29427,   103,   103,   103,   103,   103,   103,  2004,
          103, 20330,   103])

In [272]:
_token_ids_mask * (probs == 0)

tensor([103, 103,   0, 103, 103, 103, 103, 103, 103,   0, 103,   0, 103])

In [273]:
_token_ids_real * (probs == 1)

tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0, 2004,    0,    0,
           0])

In [274]:
_token_ids_rand * (probs == 2)

tensor([    0,     0, 29427,     0,     0,     0,     0,     0,     0,     0,
            0, 20330,     0])

In [353]:
"""
# 原来的
tensor([[  101,  1999,  2755,  1010,  2009,  2003,  2035,  1037,  3160,  1997,
          2576,  2097,  1010,  2500,  2031,  2056,  2023,  1998,  1045,  2052,
          2066,  2000,  6911,  1996,  2391,  1010,  2498,  2590,  2097,  4148,
          2065,  2057,  2079,  2025,  2215,  2009,  2000,  1012,   102],
        [  101,  2101,  2006,  1010,  2009,  2003,  2825,  2000,  8752, 14472,
          2030,  4961,  1011,  2039,  2336,  1999,  2048,  3584,  9620,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2023,  2003,  2055,  2004,  2172,  2004,  2057,  2064,  3246,
          2005,  2306,  1996, 25722,  1997,  1996,  2691, 13424,  3343,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]])
"""
token_ids = token_ids.masked_scatter(pred_mask, _token_ids)
token_ids

tensor([[  101,  1999,  2755,  1010,  2009,  2003,  2035,  1037,  3160,  1997,
           103,  2097,  1010,   103,  2031,  2056,  2023,  1998,  1045,  2052,
          2066,  2000,  6911,  1996,  2391,  1010,  2498,  2590,  2097, 29427,
          2065,  2057,  2079,  2025,   103,  2009,  2000,  1012,   102],
        [  101,   103,  2006,  1010,  2009,  2003,  2825,  2000,   103,   103,
          2030,  4961,  1011,  2039,   103,  1999,  2048,  3584,  9620,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2023,  2003,   103,  2004,   103,  2004,  2057, 20330,  3246,
          2005,  2306,  1996, 25722,  1997,  1996,  2691, 13424,   103,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]])

In [354]:
token_ids.shape

torch.Size([3, 39])

In [281]:
mlm_labels[~pred_mask] = -100

In [355]:
mlm_labels

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          2576,  -100,  -100,  2500,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  4148,
          -100,  -100,  -100,  -100,  2215,  -100,  -100,  -100,  -100],
        [ -100,  2101,  -100,  -100,  -100,  -100,  -100,  -100,  8752, 14472,
          -100,  -100,  -100,  -100,  2336,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [ -100,  -100,  -100,  2055,  2004,  2172,  -100,  -100,  2064,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  3343,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100]])

In [356]:
mlm_labels.shape

torch.Size([3, 39])

In [279]:
pred_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
          True, False, False,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,  True,
         False, False, False, False,  True, False, False, False, False],
        [False,  True, False, False, False, False, False, False,  True,  True,
         False, False, False, False,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [False, False, False,  True,  True,  True, False, False,  True, False,
         False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [280]:
mlm_labels

tensor([[  101,  1999,  2755,  1010,  2009,  2003,  2035,  1037,  3160,  1997,
          2576,  2097,  1010,  2500,  2031,  2056,  2023,  1998,  1045,  2052,
          2066,  2000,  6911,  1996,  2391,  1010,  2498,  2590,  2097,  4148,
          2065,  2057,  2079,  2025,  2215,  2009,  2000,  1012,   102],
        [  101,  2101,  2006,  1010,  2009,  2003,  2825,  2000,  8752, 14472,
          2030,  4961,  1011,  2039,  2336,  1999,  2048,  3584,  9620,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2023,  2003,  2055,  2004,  2172,  2004,  2057,  2064,  3246,
          2005,  2306,  1996, 25722,  1997,  1996,  2691, 13424,  3343,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]])

### train step

In [286]:
input_ids=token_ids
attention_mask=attn_mask
lm_labels=mlm_labels

In [419]:
s_logits, s_hidden_states = student(input_ids=input_ids, attention_mask=attention_mask)

In [420]:
s_logits.shape

torch.Size([3, 39, 30522])

In [421]:
len(s_hidden_states)

7

In [300]:
(lm_labels > -1).shape

torch.Size([3, 39])

In [302]:
attention_mask

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [303]:
lm_labels > -1

tensor([[False, False, False, False, False, False, False, False, False, False,
          True, False, False,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,  True,
         False, False, False, False,  True, False, False, False, False],
        [False,  True, False, False, False, False, False, False,  True,  True,
         False, False, False, False,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [False, False, False,  True,  True,  True, False, False,  True, False,
         False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [315]:
mask = (lm_labels > -1).unsqueeze(-1).expand_as(s_logits)
mask

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [ True,  True,  True,  ...,  True,  True,  True],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [

In [316]:
mask.shape

torch.Size([3, 39, 30522])

In [319]:
s_logits.shape

torch.Size([3, 39, 30522])

In [320]:
s_logits_slct = torch.masked_select(s_logits, mask)  # (bs * seq_length * voc_size) modulo the 1s in mask
s_logits_slct.shape

torch.Size([396786])

In [323]:
s_logits_slct = s_logits_slct.view(-1, s_logits.size(-1))  # (bs * seq_length, voc_size) modulo the 1s in mask

In [324]:
s_logits_slct.shape

torch.Size([13, 30522])

In [325]:
n_tgt

13

In [458]:
# if NOT self.params.restrict_ce_to_mask:
torch.masked_select(
    s_logits, attention_mask.unsqueeze(-1).expand_as(s_logits)).view(-1, s_logits.size(-1)).shape

torch.Size([81, 30522])

In [309]:
x = torch.randn(3, 4)
mask = x.ge(0.5)

In [314]:
x

tensor([[ 0.1517, -0.9983,  0.6421,  1.0643],
        [-0.2531, -0.4486, -0.2509, -0.4240],
        [ 1.0962,  0.6863,  0.3712,  0.5759]])

In [310]:
mask

tensor([[False, False,  True,  True],
        [False, False, False, False],
        [ True,  True, False,  True]])

In [313]:
torch.masked_select(x, mask)

tensor([0.6421, 1.0643, 1.0962, 0.6863, 0.5759])

In [ ]:
t_logits_slct = torch.masked_select(t_logits, mask)  # (bs * seq_length * voc_size) modulo the 1s in mask
t_logits_slct = t_logits_slct.view(-1, s_logits.size(-1))  # (bs * seq_length, voc_size) modulo the 1s in mask

In [326]:
import torch.nn.functional as F

In [330]:
x

tensor([[ 0.1517, -0.9983,  0.6421,  1.0643],
        [-0.2531, -0.4486, -0.2509, -0.4240],
        [ 1.0962,  0.6863,  0.3712,  0.5759]])

In [337]:
np.log(F.softmax(x, dim=0))

tensor([[-1.4443, -2.0946, -0.7757, -0.6095],
        [-1.8490, -1.5450, -1.6687, -2.0977],
        [-0.4997, -0.4101, -1.0466, -1.0978]])

In [336]:
F.log_softmax(x, dim=0)

tensor([[-1.4443, -2.0946, -0.7757, -0.6095],
        [-1.8490, -1.5450, -1.6687, -2.0977],
        [-0.4997, -0.4101, -1.0466, -1.0978]])

In [338]:
s_logits_slct.shape

torch.Size([13, 30522])

In [341]:
F.log_softmax(s_logits_slct / 2.0, dim=-1).shape

torch.Size([13, 30522])

In [344]:
import torch.nn as nn

In [349]:
s_logits.shape

torch.Size([3, 39, 30522])

In [ ]:
loss_mlm = lm_loss_fct(s_logits.view(-1, s_logits.size(-1)), lm_labels.view(-1))

In [350]:
s_logits.view(-1, s_logits.size(-1)).shape

torch.Size([117, 30522])

In [351]:
lm_labels.view(-1).shape

torch.Size([117])

In [352]:
lm_labels

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          2576,  -100,  -100,  2500,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  4148,
          -100,  -100,  -100,  -100,  2215,  -100,  -100,  -100,  -100],
        [ -100,  2101,  -100,  -100,  -100,  -100,  -100,  -100,  8752, 14472,
          -100,  -100,  -100,  -100,  2336,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [ -100,  -100,  -100,  2055,  2004,  2172,  -100,  -100,  2064,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  3343,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100]])

In [372]:
len(s_hidden_states)

7

In [378]:
s_hidden_states[-1].shape

torch.Size([3, 39, 768])

In [404]:
attn_mask

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [405]:
attn_mask.shape

torch.Size([3, 39])

In [423]:
mask = attention_mask.unsqueeze(-1).expand_as(s_hidden_states)

In [434]:
mask.shape

torch.Size([3, 39, 768])

In [441]:
attention_mask

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [444]:
mask[:,:,0]

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [445]:
mask[:,:,1]

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [448]:
mask[:,:,-1]

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False]])

In [449]:
lengths

tensor([39, 21, 21])

In [451]:
sum(lengths)

tensor(81)

In [422]:
s_hidden_states = s_hidden_states[-1]

In [428]:
s_hidden_states_slct = torch.masked_select(s_hidden_states, mask)

In [429]:
s_hidden_states_slct.shape

torch.Size([62208])

In [430]:
dim = s_hidden_states[-1].size(-1)
dim

768

In [431]:
s_hidden_states_slct = s_hidden_states_slct.view(-1, dim)

In [432]:
s_hidden_states_slct.shape

torch.Size([81, 768])

In [417]:
# target
s_hidden_states_slct.new(s_hidden_states_slct.size(0)).fill_(1)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [418]:
_.shape

torch.Size([81])

## 配置

In [388]:
# teacher_architecture_config = BertConfig.from_pretrained("bert-base-uncased")

In [389]:
teacher_architecture_config.num_hidden_layers

12

In [390]:
stu_architecture_config.num_hidden_layers

6